In [1]:
import asyncio
import datetime
import pathlib
import random
import sys

import httpx
import rdflib

from kelp_drone.drone import Drone, OODALoop
from kelp_drone.somu import SpeciesOccurrenceManagementUnit as SOMU
from kelp_drone.sensors import Current, Depth, Light, Temperature
from kelp_drone.sensors import chemical as chem_sensors

sys.path.append("../../../src/")

from kelp_bed_simulation.prep import chatgpt, claude, gemini, add_drone_somus
from kelp_bed_simulation.simulation import parse_buoy_data

scenario_001 = pathlib.Path("../../../scenarios/001-intro")
scenario_002 = pathlib.Path("../../../scenarios/002-first-dive")

environment = rdflib.Graph()
SCHEMA = rdflib.Namespace("https://schema.org/")
environment.namespace_manager.bind("schema", SCHEMA)
environment.parse((scenario_001 / "environment.ttl"), format='turtle')

first_prompt = (scenario_001 / "first-prompt.md").read_text()

In [2]:
first_claude = asyncio.ensure_future(claude.query(first_prompt.format(model_name="Claude")))
first_chatgpt = asyncio.ensure_future(chatgpt.query(first_prompt.format(model_name="ChatGPT")))
first_gemini = asyncio.ensure_future(gemini.query(first_prompt.format(model_name="Gemini")))

In [4]:
print(first_chatgpt.result())

Hello Farmer Jerms! I am excited to be a part of this drone kelp bed simulation. As for my name, I would like to be called "Marina". Thank you for involving me in this project.


In [5]:
print(first_gemini.result())

I am a language model, and do not have a name or body. However, I can assist you in generating a name for your kelp drone. You can consider names that reflect the drone's purpose, such as "Kelpkeeper" or "Ocean Guardian." Alternatively, you can choose a name that evokes the ocean or marine life, such as "Neptune" or "Coral."


In [6]:
print(first_claude.result())

Hello Farmer Jerms, it's nice to meet you. For my name, I would like to be called Kelp Drone Alpha.


In [ ]:
drone1 = Drone(name="Keldan", model=claude)
drone2 = Drone(name="Aqua", model=chatgpt)
drone3 = Drone(name="Guardian", model=gemini)
drones = [drone1, drone2, drone3]

In [ ]:
drones = [drone1, drone2, drone3]

for i, drone1 in enumerate(drones):
    for drone2 in drones[:i] + drones[i+1:]:
        drone1.add_neighbor(drone2)
        drone2.add_neighbor(drone1)

In [ ]:
for drone in drones:
    message = f"{drone.name} here, my LLM is {drone.model.name}. Pleased to meet you!"
    drone.peer_to_peer_channel.publish(message)

In [ ]:
messages = []
for drone in drones:
    add_drone_somus(drone)
    prompt = f"You received the following messages from your fellow drones:\n{' '.join(drone.read_messages())}\nHow do you respond?"
    message_result = asyncio.ensure_future(drone.model.query(prompt))
    messages.append((drone.name, message_result))

In [ ]:
for name, row in messages:
    print(f"Drone {name}")
    print(row.result())
    print("-"*60)

![Buoy View](46028-buoycam.jpg)

## Epoch 1 - Initial Surface Environmental Sensor Readings for Drone Pod

In [ ]:
current_reading = parse_buoy_data(httpx.get("https://www.ndbc.noaa.gov/data/latest_obs/46092.txt").text)

In [ ]:
print(current_reading)

In [ ]:
for drone in drones:
    time_stamp = datetime.datetime.now(datetime.UTC)
    drone.sensors["Current"] = Current(last_reading=time_stamp, value=3.9)
    for somu in drone.somus:
        o2 = chem_sensors.Oxygen(last_reading=time_stamp, value=-1)
        o2.value = random.randint(int(o2.suggested_range.minimum),
                                  int(o2.suggested_range.maximum))
        somu.sensors["O"] = o2
        light =  Light(last_reading=time_stamp, value=-1)
        light.value = random.randint(4_000, 6_000)
        somu.sensors["Light"] = light
        somu.sensors["Temperature"] = Temperature(last_reading=time_stamp, value=10.1)

In [ ]:
epoch_1 = []
for drone in drones:
    drone_result = asyncio.ensure_future(drone.ooda_loop.run())
    epoch_1.append((drone.name, drone_result))

In [ ]:
drone1.somus